# Inference on an upcoming dataset

In this part, we will simulate the real deployment of the package and make inference on an upcoming dataset. We use the Adult dataset from UCI datasets which has an individual testing set.

## Training models

Similar to the first example, we initialize a `Trainer` and model bases, then train all models.

In [1]:
import torch
from tabensemb.trainer import Trainer
from tabensemb.model import *
import tabensemb
from tabensemb.config import UserConfig
import os
from tempfile import TemporaryDirectory

temp_path = TemporaryDirectory()
tabensemb.setting["default_output_path"] = os.path.join(temp_path.name, "output")
tabensemb.setting["default_config_path"] = os.path.join(temp_path.name, "configs")
tabensemb.setting["default_data_path"] = os.path.join(temp_path.name, "data")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

trainer = Trainer(device=device)
cfg = UserConfig.from_uci("Adult", sep=", ")
trainer.load_config(cfg)
trainer.load_data()
models = [
    PytorchTabular(trainer, model_subset=["Category Embedding"]),
    WideDeep(trainer, model_subset=["TabMlp"]),
    AutoGluon(trainer, model_subset=["Linear Regression"]),
]
trainer.add_modelbases(models)
trainer.train(stderr_to_stdout=True)
trainer.get_leaderboard()

Using cuda device


/home/xlluo/hdd/tabular_ensemble/tabensemb/config/user_config.py:226: UserWarning: There exists .test file(s) ['adult.test'] which should be used for final metrics. The .zip file isleft for the user to process.
  warnings.warn(
/home/xlluo/hdd/tabular_ensemble/tabensemb/utils/utils.py:315: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(s), names=names, sep=sep)


Project will be saved to /tmp/tmplxwlbz1p/output/adult/2023-08-03-20-56-18-0_UserInputConfig
Dataset size: 19536 6512 6513
Data saved to /tmp/tmplxwlbz1p/output/adult/2023-08-03-20-56-18-0_UserInputConfig (data.csv and tabular_data.csv).

-------------Run PytorchTabular-------------

Training Category Embedding
Global seed set to 42
2023-08-03 20:56:20,384 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-08-03 20:56:20,385 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for classification task
2023-08-03 20:56:20,443 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-08-03 20:56:20,464 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-08-03 20:56:21,292 - {pytorch_tabular.tabular_model:582

,Program,Model,Training F1_SCORE,Training PRECISION_SCORE,Training RECALL_SCORE,Training JACCARD_SCORE,Training ACCURACY_SCORE,Training BALANCED_ACCURACY_SCORE,Training COHEN_KAPPA_SCORE,Training HAMMING_LOSS,...,Validation ACCURACY_SCORE,Validation BALANCED_ACCURACY_SCORE,Validation COHEN_KAPPA_SCORE,Validation HAMMING_LOSS,Validation MATTHEWS_CORRCOEF,Validation ZERO_ONE_LOSS,Validation ROC_AUC_SCORE,Validation LOG_LOSS,Validation BRIER_SCORE_LOSS,Validation AVERAGE_PRECISION_SCORE
0,AutoGluon,Linear Regression,0.649959,0.719298,0.592813,0.481437,0.846284,0.759732,0.552647,0.153716,...,0.841370,0.755397,0.540607,0.158630,0.544061,0.158630,0.896542,0.335764,0.108401,0.848356
1,WideDeep,TabMlp,0.694051,0.730498,0.661067,0.531453,0.859695,0.791870,0.603321,0.140305,...,0.852426,0.783820,0.584325,0.147574,0.585265,0.147574,0.908941,0.317243,0.101578,0.868427
2,PytorchTabular,Category Embedding,0.708106,0.742191,0.677015,0.548115,0.865633,0.801226,0.621075,0.134367,...,0.851044,0.780948,0.579583,0.148956,0.580653,0.148956,0.909818,0.315222,0.101306,0.869300


*Optional*: Use the following line, we can run k-fold cross-validation and check the leaderboard.

```python
trainer.get_leaderboard(cross_validation=10, split_type="cv", stderr_to_stdout=True)
```

**Remark**: `split_type` can be `random`, which means that the dataset is randomly split according to the given `split_ratio` in the configuration and different random seeds.

## Selecting and storing a model

From the leaderboard, we can check the performance of each model and select one of the models for deployment. Say we want to choose `Linear Regression` from `AutoGluon`, we detach the model from the heavy `trainer`. It is also stored locally in a separate directory.

In [2]:
trainer_of_one_model = trainer.detach_model(program="AutoGluon", model_name="Linear Regression")

Trainer saved. To load the trainer, run trainer = load_trainer(path='/tmp/tmplxwlbz1p/output/adult/2023-08-03-20-56-18-0_UserInputConfig-I1/trainer.pkl')


The detached trainer now has only one model base.

In [3]:
# Model bases of the detached trainer
trainer_of_one_model.modelbases

In [4]:
# The model in the model base
trainer_of_one_model.get_modelbase("AutoGluon_Linear Regression").model["Linear Regression"]

('Linear Regression',
 <autogluon.tabular.predictor.predictor.TabularPredictor at 0x7f09803be760>)

## Loading the model

Now the `Trainer` containing a single model stores in a seperate directory. Assume that we want to load the local trainer in a separate script for inference. In the following line, the argument `path` of `load_trainer` is the path to `trainer.pkl`, which is already printed when detaching the model or training the model bases. Here we just use the directory of the detached trainer `trainer_of_one_model`.

**Remark**: You can move the directory to any other places (or other devices if the version of the package and the environment are all consistent) and rename the folder. `tabensemb` automatically configures the path.

In [5]:
from tabensemb.trainer import load_trainer

trainer = load_trainer(path=os.path.join(trainer_of_one_model.project_root, "trainer.pkl"))

In [6]:
trainer.get_modelbase("AutoGluon_Linear Regression").model["Linear Regression"]

('Linear Regression',
 <autogluon.tabular.predictor.predictor.TabularPredictor at 0x7f0954ba9fd0>)

## Inference

Assume that we have a new `DataFrame` representing an upcoming dataset. For demonstration, we use the testing set here. The classification target is ordinal encoded by `trainer.datamodule.label_ordinal_encoder`.

In [7]:
df = trainer.df.loc[trainer.test_indices, :]
truth = trainer.df.loc[trainer.test_indices, trainer.label_name].values.flatten()
truth

array([0, 0, 1, ..., 1, 0, 0])

Use the functionality of the model base to do inference. The returned result should be

In [8]:
import pandas as pd

result = trainer.get_modelbase("AutoGluon_Linear Regression").predict(df, model_name="Linear Regression")
result

array([['<=50K'],
       ['<=50K'],
       ['>50K'],
       ...,
       ['>50K'],
       ['<=50K'],
       ['<=50K']], dtype=object)

You can see the F1 score on the "new" (testing) dataset is the same as that in the above leaderboard. The result should be ordinal-encoded first to calculate metrics. We provide `DataModule.label_categories_transform` to achieve this (and `DataModule.label_categories_inverse_transform` to do inverse transform).

The `auto_metric_sklearn` automatically calculates different kinds of `sklearn.metrics`, which is extremely useful for classification tasks

In [9]:
from tabensemb.utils import auto_metric_sklearn

encoded_result = trainer.datamodule.label_categories_transform(pd.DataFrame(result, columns=trainer.label_name)).values
# You can also use trainer.datamodule.label_ordinal_encoder.transform(result) to get the same result.
auto_metric_sklearn(truth, encoded_result, "f1_score", "binary"), trainer.leaderboard.loc[trainer.leaderboard["Model"]=="Linear Regression", "Testing F1_SCORE"].values

(0.6615652173913042, array([0.66156522]))

## Inference on the individual testing set

When loading from UCI datasets, `UserConfig.from_uci` finds that a testing data exists, so the downloaded .zip file is not removed. We can load the archive using `zipfile`.

In [10]:
import zipfile

zipf = zipfile.ZipFile(os.path.join(tabensemb.setting["default_data_path"], "Adult.zip"))
zipf.namelist()

['Index', 'adult.data', 'adult.names', 'adult.test', 'old.adult.names']

Now check the content of `adult.text`. It is a .csv-like file, just like the `adult.data` file but has an additional row at the front, and an additional "." at the end of each line.

In [11]:
file = zipf.read("adult.test").decode()
print(file[:500])

|1x3 Cross validator
25, Private, 226802, 11th, 7, Never-married, Machine-op-inspct, Own-child, Black, Male, 0, 0, 40, United-States, <=50K.
38, Private, 89814, HS-grad, 9, Married-civ-spouse, Farming-fishing, Husband, White, Male, 0, 0, 50, United-States, <=50K.
28, Local-gov, 336951, Assoc-acdm, 12, Married-civ-spouse, Protective-serv, Husband, White, Male, 0, 0, 40, United-States, >50K.
44, Private, 160323, Some-college, 10, Married-civ-spouse, Machine-op-inspct, Husband, Black, Male, 7688, 0


We provide the `str_to_dataframe` function that translate the string object to a `DataFrame` and replace illegal values by `np.nan`.

In [12]:
from tabensemb.utils import str_to_dataframe

file = file.replace("|1x3 Cross validator\n","").replace(".\n", "\n")
testing_df = str_to_dataframe(file, sep=", ", names=trainer.df.columns, check_nan_on=trainer.cont_feature_names)
testing_df

/home/xlluo/hdd/tabular_ensemble/tabensemb/utils/utils.py:315: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(s), names=names, sep=sep)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


Inference is similar.

In [13]:
result = trainer.get_modelbase("AutoGluon_Linear Regression").predict(testing_df, model_name="Linear Regression")
result

array([['<=50K'],
       ['<=50K'],
       ['<=50K'],
       ...,
       ['>50K'],
       ['<=50K'],
       ['>50K']], dtype=object)

Both the truth and the result require to be ordinal-encoded to calculate metrics.

In [14]:
encoded_truth = trainer.datamodule.label_categories_transform(testing_df[trainer.label_name]).values
encoded_result = trainer.datamodule.label_categories_transform(pd.DataFrame(result, columns=trainer.label_name)).values
auto_metric_sklearn(encoded_truth, encoded_result, "f1_score", "binary"), auto_metric_sklearn(encoded_truth, encoded_result, "roc_auc_score", "binary")

(0.6438414547520955, 0.7573179597871491)